# Data collection & Preparation

# 1.Import python package

In [10]:
import pandas as pd
import numpy as np
import os
file_path='C:/Business_Data_Analysis/data/'

In [11]:
%pip install pypandoc --user

Note: you may need to restart the kernel to use updated packages.


# 2. Establish connection to database

In [12]:
import pyodbc
import urllib
import sqlalchemy

'''sources databases'''
params_p = urllib.parse.quote_plus("DRIVER={SQL Server Native Client 11.0};"
                                  "SERVER=localhost\SQLEXPRESS;"
                                  "DATABASE=DataWarehouse1;"
                                  "UID=sa;"
                                  "PWD=user1")

engine_p = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect={}".format(params_p))

params_s = urllib.parse.quote_plus("DRIVER={SQL Server Native Client 11.0};"
                                  "SERVER=localhost\SQLEXPRESS;"
                                  "DATABASE=DataWarehouse2;"
                                  "UID=sa;"
                                  "PWD=user1")

engine_s = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect={}".format(params_s))

'''destination databases'''

params_datahub = urllib.parse.quote_plus("DRIVER={SQL Server Native Client 11.0};"
                                  "SERVER=localhost\SQLEXPRESS;"
                                  "DATABASE=datahub;"
                                  "UID=sa;"
                                  "PWD=user1")

engine_datahub = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect={}".format(params_datahub))

# 3. Back up your data

In [13]:
# '''get historical data from source database'''
# df_sales_p=pd.read_excel('sales_data_primary',engine_p)

# from datetime import datetime
# dateTimeObj = datetime.now()
# timestampStr = dateTimeObj.strftime('%d-%m-%Y-%H-%M-%S')
# print('Current Timestamp:',timestampStr)

# df_sales_p.to_csv(file_path+'data/sales_data_primary_backup_'+timestampStr+'.csv',index=False)

# 4. ETL data to datahub

In [30]:
df_UCI=pd.read_excel(file_path+'UCI Case Data.xlsx')
df_UCI.to_sql('UCI',engine_datahub,if_exists='replace',index=False)

In [39]:
df_UCI.shape

(624, 16)

# i. Append new data to historical data

Using Python

In [40]:
from datetime import datetime

'''get new data'''
df_T2UCI_new=pd.read_excel(file_path+'UCI Case Data_new.xlsx')

In [41]:
df_T2UCI_new.shape

(24, 16)

In [42]:
'''get historical data from source database'''
df_UCI=pd.read_sql('UCI',engine_datahub)

In [43]:
df_UCI.shape

(624, 16)

In [44]:
'''append new data to historical data'''
df_UCI=pd.concat([df_UCI,df_T2UCI_new])

In [45]:
df_UCI.shape

(648, 16)

In [46]:
'''drop duplicate'''
df_UCI.drop_duplicates(keep='first', inplace=True)

In [47]:
df_UCI.shape

(648, 16)

# ii. Check data

In [48]:
df_T2UCI_new.shape,df_UCI.shape

((24, 16), (648, 16))

# iii. Export updated historical data to datebase

In [49]:
df_UCI.to_sql('UCI',engine_datahub,if_exists='replace',index=False)